In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### GraphSAGE(pytorch-version)

#### import Module 

In [3]:
import torch
print(torch.__version__)

1.13.0+cu116


In [4]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 1.9 MB 12.9 MB/s 
     |████████████████████████████████| 9.4 MB 79.5 MB/s 
     |████████████████████████████████| 4.6 MB 95.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 4.3 MB/s 
     |████████████████████████████████| 280 kB 92.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=e1bd3cf0e8cc9291e8abeee0d972e511e36e05e341da9d3ea296d408a415b7ba
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstal

#### configuration

In [5]:
import logging
import pickle
import torch

def make_logger(name=None):
    # https://hwangheek.github.io/2019/python-logging/
    logger = logging.getLogger(name) # 로거 객체를 생성
    logger.setLevel(logging.DEBUG) # 로그의 레벨
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(message)s") # LogRecord의 출력 형태 지정

    # Handler는 로그 메시지를 출력하는 역할
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    console.setFormatter(formatter)

    logger.addHandler(console)
    return logger

def dump_pickle(address, file):
    with open(address, 'wb') as f:
        pickle.dump(file, f)

def load_pickle(address):
    with open(address, 'rb') as f:
        data = pickle.load(f)
    return data

class EarlyStopping:
    def __init__(self, patience=10, verbose=False, save_path='checkpoint.pt'):
        """
        :param patience: how many times you will wait before earlystopping
        :param save_path: where to save checkpoint
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.save_path = save_path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model, val_loss)
        elif score < self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model, val_loss)
            self.counter = 0 # reset

    def save_checkpoint(self, model, val_loss):
        if self.verbose:
            print(f"val loss: ({self.val_loss_min:.6f} -> {val_loss:.6f})")
        torch.save(model.state_dict(), self.save_path)
        self.val_loss_min = val_loss

In [6]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.datasets import Reddit
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv

logger = make_logger(name='graphsage_logger')

In [7]:
%cd gdrive/MyDrive/GraphSAGE

[Errno 2] No such file or directory: 'gdrive/MyDrive/GraphSAGE'
/content


In [8]:
# Load Reddit Dataset
import os
path = os.path.join(os.getcwd(),'data','Reddit')

dataset = Reddit(path)
data = dataset[0]

Extracting /content/data/Reddit/raw/reddit.zip
Processing...
Done!


In [9]:
print(data)

Data(x=[232965, 602], edge_index=[2, 114615892], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965])


In [10]:
# Data verification
logger.info(f"Node Feature MAtrix InFo : # Nodes: {data.x.shape[0]}, # Node Features : {data.x.shape[1]}")

# Edge index
# Graph Connectivity in Coo Format with shape (2, num_edges)
logger.info(f"Edge index shape : {data.edge_index.shape}")
logger.info(f"Edge weight: {data.edge_attr}")

# train_mask denotes against which nodes to train
print(len(data.train_mask))
print(data.train_mask.sum())

2022-12-13 06:38:00,371 - graphsage_logger - Node Feature MAtrix InFo : # Nodes: 232965, # Node Features : 602
INFO:graphsage_logger:Node Feature MAtrix InFo : # Nodes: 232965, # Node Features : 602
2022-12-13 06:38:00,373 - graphsage_logger - Edge index shape : torch.Size([2, 114615892])
INFO:graphsage_logger:Edge index shape : torch.Size([2, 114615892])
2022-12-13 06:38:00,376 - graphsage_logger - Edge weight: None
INFO:graphsage_logger:Edge weight: None


232965
tensor(153431)


In [11]:
# Define Sampler
train_loader = NeighborSampler(
    data.edge_index, node_idx=data.train_mask,
    sizes=[25, 10], batch_size=1024, shuffle=True, num_workers=12)

subgraph_loader = NeighborSampler(
    data.edge_index, node_idx=None,
    sizes=[-1], batch_size=1024, shuffle=False, num_workers=12)


/usr/local/lib/python3.8/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:
print(train_loader)
print(subgraph_loader)

NeighborSampler(sizes=[25, 10])
NeighborSampler(sizes=[-1])


In [13]:
next(iter(train_loader))

(1024,
 tensor([ 56763, 156564, 104494,  ...,  80749, 195147, 146918]),
 [EdgeIndex(edge_index=tensor([[10866, 10878, 19709,  ..., 21668, 21671, 21672],
          [    0,     0,     0,  ..., 21673, 21673, 21673]]), e_id=tensor([86646886, 34334275, 76882858,  ..., 88386657,  9981250,  7102880]), size=(106277, 21674)),
  EdgeIndex(edge_index=tensor([[  457,  1024,  1025,  ..., 21671, 21672, 21673],
          [    0,     0,     0,  ...,  1023,  1023,  1023]]), e_id=tensor([20415259, 43538122, 51235531,  ...,  9978656,  7102819, 35764977]), size=(21674, 1024))])

In [14]:
# Look
batch_size, n_id, adjs = next(iter(train_loader))

# 1) batch_size
# 현재 batch size를 의미함 (integer)
logger.info(f"Current Batch Size: {batch_size}")

# 2) n_id
# 이번 Subgraph에서 사용된 모든 node id
# batch_size개의 Row를 예측하기 위해서 이에 대한 1차 이웃 node A개가 필요하고
# 1차 이웃 node A개를 위해서는 2차 이웃 node B개가 필요함
# n_id.shape = batch_size + A + B
logger.info(f"현재 Subgraph에서 사용된 모든 node id의 개수: {n_id.shape[0]}")

# 3) adjs
# 아래와 같이 Layer의 수가 2개이면 adjs는 길이 2의 List가 된다.
# head node가 있고 1-hop neighbors와 2-hop neighbors가 있다고 할 때
# adjs[1]이 head node와 1-hop neighbors의 관계를 설명하며  (1번째 Layer)
# adjs[0]이 1-hop neighbors와 2-hop neighbors의 관계를 설명한다. (2번째 Layer)
logger.info(f"Layer의 수: {len(adjs)}")

# 각 리스트에는 아래와 같은 튜플이 들어있다.
# (edge_index, e_id, size)
# edge_index: source -> target nodes를 기록한 bipartite edges
# e_id: 위 edge_index에 들어있는 index가 Full Graph에서 갖는 node id

# size: 위 edge_index에 들어있는 node의 수를 튜플로 나타낸 것으로
# head -> 1-hop 관계를 예시로 들면,
# head node의 수가 a개, 1-hop node의 수가 b개라고 했을 때
# size = (a+b, a)
# 또한 target node의 경우 source nodes의 리스트의 시작 부분에 포함되어 있어
# skip-connections나 self-loops를 쉽게 사용할 수 있게 되어 있음
A = adjs[1].size[0] - batch_size
B = adjs[0].size[0] - A - batch_size

logger.info(f"진행 방향: {B}개의 2-hop neighbors ->"
            f"{A}개의 1-hop neighbors -> {batch_size}개의 Head Nodes")

2022-12-13 06:38:39,666 - graphsage_logger - Current Batch Size: 1024
INFO:graphsage_logger:Current Batch Size: 1024
2022-12-13 06:38:39,669 - graphsage_logger - 현재 Subgraph에서 사용된 모든 node id의 개수: 107034
INFO:graphsage_logger:현재 Subgraph에서 사용된 모든 node id의 개수: 107034
2022-12-13 06:38:39,671 - graphsage_logger - Layer의 수: 2
INFO:graphsage_logger:Layer의 수: 2
2022-12-13 06:38:39,675 - graphsage_logger - 진행 방향: 85156개의 2-hop neighbors ->20854개의 1-hop neighbors -> 1024개의 Head Nodes
INFO:graphsage_logger:진행 방향: 85156개의 2-hop neighbors ->20854개의 1-hop neighbors -> 1024개의 Head Nodes


In [15]:
print(adjs)

[EdgeIndex(edge_index=tensor([[  1024,   1044,   1203,  ..., 107031, 107032, 107033],
        [     0,      1,      1,  ...,  21877,  21877,  21877]]), e_id=tensor([ 22854837, 103644544,  74461702,  ...,   9388556,  50869465,
         34961078]), size=(107034, 21878)), EdgeIndex(edge_index=tensor([[ 1024,  1025,  1026,  ..., 21875, 21876, 21877],
        [    0,     1,     1,  ...,  1023,  1023,  1023]]), e_id=tensor([ 22854837, 113115088,  37512591,  ...,  71936222,  12968253,
        101202791]), size=(21878, 1024))]


In [16]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [17]:
# Define Model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()

        self.num_layers = 2

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)

            # 마지막 Layer는 Dropout을 적용하지 않는다.
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch.
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in subgraph_loader:
                edge_index, _, size = adj.to(device)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x.cpu())

                pbar.update(batch_size)

            x_all = torch.cat(xs, dim=0)

        pbar.close()
        return x_all


In [18]:
model = GraphSAGE(dataset.num_features, 256, dataset.num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x = data.x.to(device)
y = data.y.squeeze().to(device)


In [19]:

def train(epoch):
    model.train()

    pbar = tqdm(total=int(data.train_mask.sum()))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / int(data.train_mask.sum())
    return loss, approx_acc


@torch.no_grad()
def test():
    model.eval()
    out = model.inference(x)

    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    results = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
    return results


In [21]:

for epoch in range(1, 31):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    train_acc, val_acc, test_acc = test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch 01: 100%|██████████| 153431/153431 [00:16<00:00, 9131.56it/s] 


Epoch 01, Loss: 0.5766, Approx. Train: 0.9314


Evaluating: 100%|██████████| 465930/465930 [00:30<00:00, 15515.94it/s]


Train: 0.9634, Val: 0.9519, Test: 0.9508


Epoch 02: 100%|██████████| 153431/153431 [00:16<00:00, 9240.10it/s] 


...

Train: 0.9743, Val: 0.9527, Test: 0.9521


Epoch 30: 100%|██████████| 153431/153431 [00:16<00:00, 9078.52it/s] 


Epoch 30, Loss: 0.5294, Approx. Train: 0.9423


Evaluating: 100%|██████████| 465930/465930 [00:30<00:00, 15374.66it/s]

Train: 0.9748, Val: 0.9527, Test: 0.9533
